In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from lewidi_lib import (
    enable_logging,
    preds_file,
    process_rdf_and_add_perf_metrics,
)

enable_logging()

file = preds_file(
    dataset="CSC",
    split="train",
    template="31",
    model_id="Qwen/Qwen3-32B",
    run_name="allex_20loops",
    format="parquet",
)
assert file.exists(), file.absolute()
rdf = pd.read_parquet(file)
rdf = rdf.query("run_idx <= 9")
rdf = process_rdf_and_add_perf_metrics(rdf, discard_invalid_pred=True)
rdf.drop_duplicates(subset=["dataset_idx", "run_idx"], inplace=True)

In [ ]:
len(rdf)

In [ ]:
from lewidi_lib import avg_pairwise_ws_loss

answer_diversity = rdf.groupby("dataset_idx", as_index=False).agg(
    avg_pairwise_ws_loss=("pred", avg_pairwise_ws_loss),
    avg_ws_loss=("ws_loss", "mean"),
)

In [ ]:
from lewidi_lib import compute_average_baseline_and_assing_perf_metrics


model_avg_rdf = compute_average_baseline_and_assing_perf_metrics(rdf)
model_avg_rdf = model_avg_rdf[["dataset_idx", "ws_loss"]].rename(
    columns={"ws_loss": "model_avg_ws_loss"}
)

In [ ]:
from lewidi_lib import assign_col_diversity
answer_diversity = assign_col_diversity(answer_diversity)
joint = answer_diversity.merge(model_avg_rdf, on="dataset_idx", how="left")
joint = joint.assign(improvement=lambda df: df["avg_ws_loss"] - df["model_avg_ws_loss"])

In [ ]:
from pathlib import Path
import seaborn as sns

sns.set_context("talk")

grid = sns.JointGrid(data=joint, x="avg_pairwise_ws_loss", y="improvement")
grid.plot_joint(sns.scatterplot, data=joint, alpha=0.2)
grid.plot_joint(sns.regplot, scatter=False, lowess=True)
grid.plot_marginals(sns.histplot, data=joint)
grid.ax_joint.grid(alpha=0.5)
grid.set_axis_labels(xlabel="Answer Diversity", ylabel="Model Averaging Improvement")
tgt_dir = Path("./imgs/diversity")
tgt_dir.mkdir(parents=True, exist_ok=True)
grid.figure.savefig(
    tgt_dir / "model-avg-improvement-vs-answer-diversity.pdf", bbox_inches="tight"
)

In [ ]:
axs = sns.barplot(
    joint,
    x="diversity",
    y="improvement",
)
axs.set_xlabel("Answer Diversity")
axs.set_ylabel("Model Averaging\nImprovement")
axs.grid(alpha=0.5, axis="y")
axs.figure.savefig(tgt_dir / "model-avg-improvement-vs-answer-diversity-quantiles.pdf", bbox_inches="tight")

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(figsize=(12, 4), ncols=2, gridspec_kw={"wspace": 0.3})
ax1, ax2 = axs

sns.boxplot(
    rdf.merge(
        answer_diversity[["dataset_idx", "diversity"]], on="dataset_idx", how="left"
    ),
    x="diversity",
    y="ws_loss",
    showfliers=False,
    ax=ax1,
    whis=(5, 95),
)
ax1.set_ylabel("Simple")

sns.boxplot(
    joint,
    x="diversity",
    y="model_avg_ws_loss",
    showfliers=False,
    ax=ax2,
    whis=(5, 95),
)
ax2.set_ylabel("Model Averaging")

for ax in axs:
    ax.grid(alpha=0.5, axis="y")
    ax.set_ylim(-0.1, 3)

# What is the Worst Case Performance By Diversity?

In [ ]:
oracle = rdf.loc[rdf.groupby("dataset_idx")["ws_loss"].idxmin()]
oracle = oracle.merge(
    answer_diversity[["dataset_idx", "diversity"]], on="dataset_idx", how="left"
)

In [ ]:
ax = sns.boxplot(
    oracle,
    x="diversity",
    y="ws_loss",
    showfliers=False,
    whis=(5, 95),
)
ax.grid(alpha=0.5, axis="y")
ax.set_ylim(None, 3)
ax.set_title("BoN Oracle WS Loss")

# What is the Range For Improvement from repeated Sampling?

In [ ]:
wsloss_improv = rdf[["dataset_idx", "ws_loss"]].merge(
    oracle.rename(columns={"ws_loss": "ws_loss_best"})[["dataset_idx", "ws_loss_best"]],
    on="dataset_idx",
    how="left",
)
wsloss_improv = wsloss_improv.assign(
    improvement=lambda df: df["ws_loss"] - df["ws_loss_best"]
)
wsloss_improv = wsloss_improv.merge(
    answer_diversity[["dataset_idx", "diversity"]], on="dataset_idx", how="left"
)

fig, axs = plt.subplots(figsize=(12, 4), ncols=2, gridspec_kw={"wspace": 0.3})
ax1, ax2 = axs

sns.boxplot(
    joint,
    x="diversity",
    y="improvement",
    showfliers=False,
    whis=(5, 95),
    ax=ax1,
)
ax1.set_title("Model Averaging")

sns.boxplot(
    wsloss_improv,
    x="diversity",
    y="improvement",
    showfliers=False,
    whis=(5, 95),
    ax=ax2,
)
ax2.set_title("BoN Oracle")

for ax in axs:
    ax.grid(alpha=0.5, axis="y")
    ax.set_ylim(-0.1, 2.3)
    ax.set_xlabel("Answer Diversity")
    ax.set_ylabel("Improvement")